In [8]:
import os
import pickle
import mlflow
from scipy.sparse import csr_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [9]:
# Set up MLflow tracking
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("random-forest-train")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/mlruns/1', creation_time=1737927624570, experiment_id='1', last_update_time=1737927624570, lifecycle_stage='active', name='random-forest-train', tags={}>

In [10]:
def load_pickle(filename: str):
    """Load data from a pickle file."""
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [ ]:
## TRAIN

In [11]:
def run_train(data_path: str):
    """Train a Random Forest model and log the process with MLflow."""
    # Enable MLflow autologging
    mlflow.sklearn.autolog()
    
    # Load training and validation data
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    # Convert data to sparse matrices
    X_train = csr_matrix(X_train)
    X_val = csr_matrix(X_val)

    # Start an MLflow run
    with mlflow.start_run():
        # Initialize and train the model
        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        
        # Make predictions and calculate RMSE
        y_pred = rf.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)  # Compute MSE
        rmse = mse ** 0.5  # Compute RMSE

        print(f"Validation RMSE: {rmse}")

# Run the training process
data_path = "./output"  # Update this path as needed

In [12]:
run_train(data_path)

/tmp/ipykernel_72259/1612602163.py:4: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  return pickle.load(f_in)
2025/01/26 22:23:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


Validation RMSE: 5.431162180141208


In [ ]:
## HyperParameters

In [14]:
import os
import pickle
import mlflow
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Set up MLflow tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")


def load_pickle(filename: str):
    """Load data from a pickle file."""
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run_optimization(data_path: str, num_trials: int):
    """Run hyperparameter optimization for Random Forest using Hyperopt."""
    # Load training and validation data
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    def objective(params):
        """Objective function to minimize."""
        with mlflow.start_run():
            mlflow.log_params(params)
            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_val)
            mse = mean_squared_error(y_val, y_pred)
            rmse = mse ** 0.5
            mlflow.log_metric("rmse", rmse)
        return {'loss': rmse, 'status': STATUS_OK}

    # Define the search space
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }

    # Perform hyperparameter optimization
    rstate = np.random.default_rng(42)  # For reproducible results
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=Trials(),
        rstate=rstate
    )


# Run the optimization directly in Jupyter
data_path = "./output"  # Update this path if needed
num_trials = 15
run_optimization(data_path, num_trials)

  0%|                                                                                                                                 | 0/15 [00:00<?, ?trial/s, best loss=?]

/tmp/ipykernel_72259/1874926329.py:18: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  return pickle.load(f_in)
2025/01/26 22:25:59 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'



🏃 View run valuable-cub-120 at: http://127.0.0.1:5000/#/experiments/2/runs/286757682e2845ffbdac6ce03bd35c1d                                                                 

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                                                                                                                 

  7%|██████▉                                                                                                 | 1/15 [00:11<02:39, 11.39s/trial, best loss: 5.370086069268862]

2025/01/26 22:26:10 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'



🏃 View run wise-mule-774 at: http://127.0.0.1:5000/#/experiments/2/runs/f81b033ec20d4148b1088ab68929eb09                                                                    

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                                                                                                                 

 13%|█████████████▊                                                                                          | 2/15 [00:15<01:29,  6.89s/trial, best loss: 5.370086069268862]

...

🏃 View run able-shrew-869 at: http://127.0.0.1:5000/#/experiments/2/runs/eed6d6bd32134968b6f07c4fdfaf75c3                                                                   

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                                                                                                                 

 87%|█████████████████████████████████████████████████████████████████████████████████████████▎             | 13/15 [01:38<00:12,  6.48s/trial, best loss: 5.335419588556921]

2025/01/26 22:27:37 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'



🏃 View run learned-tern-929 at: http://127.0.0.1:5000/#/experiments/2/runs/3a3a80980dd1456b84cff051ac087c1c                                                                 

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                                                                                                                 

 93%|████████████████████████████████████████████████████████████████████████████████████████████████▏      | 14/15 [01:44<00:06,  6.53s/trial, best loss: 5.335419588556921]

2025/01/26 22:27:44 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'



🏃 View run ambitious-flea-27 at: http://127.0.0.1:5000/#/experiments/2/runs/28c151c6234f483fbd57ca59faa53482                                                                

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                                                                                                                 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [01:53<00:00,  7.57s/trial, best loss: 5.335419588556921]


In [ ]:
# Register

In [24]:
import os
import pickle
import mlflow
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
EXPERIMENT_NAME = "random-forest-best-models"
RF_PARAMS = ['max_depth', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 'random_state']

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.sklearn.autolog()


def load_pickle(filename):
    """Load a pickle file."""
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def train_and_log_model(data_path, params):
    """Train a Random Forest model and log its metrics with MLflow."""
    # Load data
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))
    X_test, y_test = load_pickle(os.path.join(data_path, "test.pkl"))

    # Ensure parameters are correctly typed
    typed_params = {
        'n_estimators': int(params.get('n_estimators', 100)),
        'max_depth': int(params.get('max_depth', None)) if params.get('max_depth') != 'None' else None,
        'min_samples_split': int(params.get('min_samples_split', 2)),
        'min_samples_leaf': int(params.get('min_samples_leaf', 1)),
        'random_state': int(params.get('random_state', 42)),
        'bootstrap': params.get('bootstrap', 'True') == 'True'  # Convert string to boolean
    }

    with mlflow.start_run():
        # Train the model
        rf = RandomForestRegressor(**typed_params)
        rf.fit(X_train, y_train)

        # Evaluate model on validation and test sets
        val_mse = mean_squared_error(y_val, rf.predict(X_val))
        val_rmse = val_mse ** 0.5
        mlflow.log_metric("val_rmse", val_rmse)

        test_mse = mean_squared_error(y_test, rf.predict(X_test))
        test_rmse = test_mse ** 0.5
        mlflow.log_metric("test_rmse", test_rmse)


def run_register_model(data_path, top_n):
    """Register the best Random Forest model in MLflow."""
    client = MlflowClient()

    # Retrieve the top_n model runs from the hyperparameter optimization experiment
    experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
    runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=top_n,
        order_by=["metrics.rmse ASC"]
    )

    for run in runs:
        train_and_log_model(data_path=data_path, params=run.data.params)

    # Select the model with the lowest test RMSE
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
    best_run = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1,
        order_by=["metrics.test_rmse ASC"]
    )[0]

    # Register the best model in MLflow
    run_id = best_run.info.run_id
    model_uri = f"runs:/{run_id}/model"
    mlflow.register_model(model_uri, name="rf-best-model")


# Specify parameters directly in the notebook
data_path = "./output"  # Path to the processed data
top_n = 5  # Number of top models to evaluate

# Run the main function
run_register_model(data_path, top_n)

/tmp/ipykernel_72259/1048469583.py:21: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  return pickle.load(f_in)
2025/01/26 22:52:25 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


🏃 View run redolent-elk-798 at: http://127.0.0.1:5000/#/experiments/3/runs/91c8d16d1c7b4d84bb7fc14f06972435
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


/tmp/ipykernel_72259/1048469583.py:21: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  return pickle.load(f_in)
2025/01/26 22:52:33 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


🏃 View run gifted-lamb-202 at: http://127.0.0.1:5000/#/experiments/3/runs/96743d203b544e0eb415ad7191d05623
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


/tmp/ipykernel_72259/1048469583.py:21: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  return pickle.load(f_in)
2025/01/26 22:52:42 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


...

/tmp/ipykernel_72259/1048469583.py:21: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  return pickle.load(f_in)
2025/01/26 22:52:56 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


🏃 View run omniscient-quail-602 at: http://127.0.0.1:5000/#/experiments/3/runs/9a8d4b10246f469399c19d5e2b03e482
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


/tmp/ipykernel_72259/1048469583.py:21: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  return pickle.load(f_in)
2025/01/26 22:53:10 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
Registered model 'rf-best-model' already exists. Creating a new version of this model...
2025/01/26 22:53:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: rf-best-model, version 2


🏃 View run respected-croc-195 at: http://127.0.0.1:5000/#/experiments/3/runs/3889d3cd2f354afdb531475e8b3ede93
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


Created version '2' of model 'rf-best-model'.
